# Checking lab data

In [ ]:
import os
import sys
from functools import reduce
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import ipywidgets as widgets

sys.path.insert(0, '../../')
from brickstudy import foundation

In [ ]:
lab_data = pd.read_csv('BRICK_lab_data_update_18_03_2024.csv', sep= ';', encoding='latin1')


In [ ]:
lab_data.head(3)

In [ ]:
lab_data['studieID'].unique()

In [ ]:
len(lab_data['studieID'].unique())

## So we have 47 patients, this will be extremely important later!
Below we will reformat the studieID to be in the same format as patient ID, i.e. with a _ instead of a -

In [ ]:
lab_data['studieID'] = lab_data['studieID'].astype(str)
lab_data['studieID'] = lab_data['studieID'].str.replace("-", "_")

In [ ]:
lab_data.head(3)

## We want in theory all the patients on thier own line, but not many lines per patient
We will try to accomplish this by splitting the dataset into each code-code-original i.e. each type of lab taken on the assumption we will almost always have one ;ab

In [ ]:
lab_data.code_code_original.unique()

Notice there is a nan code. This will not work out well for us. It turns out it is in this dataset always teh case with Natrium. We will make the code, a string

In [ ]:
codes = lab_data.code_code_original.unique().astype(str)
codes

We will split off the dataframes, then check to make sure we get something for every code

In [ ]:
list_of_data_frames = []
for code in codes:
    prenames = ["code_code_original", code]
    frame_name = "_".join(prenames)
    #print(frame_name)
    frame_name = lab_data[lab_data['code_code_original'] == code]
    frame_name = frame_name.add_suffix(code)
    frame_name = frame_name.rename(columns= {frame_name.columns[0]:'studieID'})
    list_of_data_frames.append(frame_name)
    

In [ ]:
len(list_of_data_frames)

In [ ]:
len(codes)

Let's look at an example:

In [ ]:
list_of_data_frames[1].head(3) 

In [ ]:
list_of_data_frames[1].shape

So far so good, we have a frame with in the example above the DBIL. We then tried to do a big reduction; but it produced repeaters

In [ ]:
#final_df = reduce(lambda left,right: pd.merge(left,right, on=['studieID'], how='outer'), list_of_data_frames)

We needed to examine why this happened so we took a look by hand:

In [ ]:
final_df2 = list_of_data_frames[0].merge(list_of_data_frames[1], on=['studieID'], how='outer')

In [ ]:
final_df2 = final_df2.merge(list_of_data_frames[3], on=['studieID'], how='outer').merge(list_of_data_frames[4], on=['studieID'], how='outer')

In [ ]:
final_df2 = final_df2.merge(list_of_data_frames[5], on=['studieID'], how='outer').merge(list_of_data_frames[6], on=['studieID'], how='outer')

In [ ]:
final_df2 = final_df2.merge(
    list_of_data_frames[7], on=['studieID'], how='outer'
).merge(list_of_data_frames[8], on=['studieID'], how='outer'
).merge(list_of_data_frames[9], on=['studieID'], how='outer'
).merge(list_of_data_frames[10], on=['studieID'], how='outer'
).merge(list_of_data_frames[11], on=['studieID'], how='outer'
).merge(list_of_data_frames[12], on=['studieID'], how='outer'
).merge(list_of_data_frames[13], on=['studieID'], how='outer'
).merge(list_of_data_frames[14], on=['studieID'], how='outer'
).merge(list_of_data_frames[15], on=['studieID'], how='outer'
).merge(list_of_data_frames[16], on=['studieID'], how='outer'
).merge(list_of_data_frames[17], on=['studieID'], how='outer'
).merge(list_of_data_frames[18], on=['studieID'], how='outer'
).merge(list_of_data_frames[19], on=['studieID'], how='outer'
).merge(list_of_data_frames[20], on=['studieID'], how='outer'
).merge(list_of_data_frames[21], on=['studieID'], how='outer'
).merge(list_of_data_frames[22], on=['studieID'], how='outer'
).merge(list_of_data_frames[23], on=['studieID'], how='outer'
# ).merge(list_of_data_frames[24], on=['studieID'], how='outer'
# ).merge(list_of_data_frames[25], on=['studieID'], how='outer'
# ).merge(list_of_data_frames[26], on=['studieID'], how='outer'
# ).merge(list_of_data_frames[27], on=['studieID'], how='outer'
# ).merge(list_of_data_frames[28], on=['studieID'], how='outer'
# ).merge(list_of_data_frames[29], on=['studieID'], how='outer'
# ).merge(list_of_data_frames[30], on=['studieID'], how='outer'
# ).merge(list_of_data_frames[31], on=['studieID'], how='outer'
# ).merge(list_of_data_frames[32], on=['studieID'], how='outer'
# ).merge(list_of_data_frames[33], on=['studieID'], how='outer'
# ).merge(list_of_data_frames[34], on=['studieID'], how='outer'
# ).merge(list_of_data_frames[35], on=['studieID'], how='outer'
# ).merge(list_of_data_frames[36], on=['studieID'], how='outer'
# ).merge(list_of_data_frames[37], on=['studieID'], how='outer'
# ).merge(list_of_data_frames[38], on=['studieID'], how='outer'
# ).merge(list_of_data_frames[39], on=['studieID'], how='outer'
       )

After some thought and examination we realized there are repeaters, because on some codes, there are multiple examples.
There is really no other way to organize ALL the data without repeaters.


In [ ]:
final_df2.tail(7)

Let's look at a repeater example:

In [ ]:
final_df2[final_df2['studieID'] == 'BRICK_055']

In this case we got lucky and we can just scroll right, and see the problem. The RDW is repeated, one venous one capillary. There is
another repeater or differing value as well, and we can tell because this is doubled

So we could at least detect the repeaters...remember it wasn't every lab with this problem, then the scientists must decide what to do about all this

In [ ]:
list_of_data_frames[0]['studieID'].is_unique
#list_of_data_frames[0]

In [ ]:
# let's do our original frame maker with an artifact
list_of_data_frames = []
dupe_q = []
for code in codes:
    prenames = ["code_code_original", code]
    frame_name = "_".join(prenames)
    #print(frame_name)
    frame_name = lab_data[lab_data['code_code_original'] == code]
    frame_name = frame_name.add_suffix(code)
    frame_name = frame_name.rename(columns= {frame_name.columns[0]:'studieID'})
    codes_dupe_list = code, frame_name['studieID'].is_unique
    print(codes_dupe_list)
    dupe_q.append(codes_dupe_list)
    list_of_data_frames.append(frame_name)

OK, but now we want to just get an actionable list of where it is wrong

In [ ]:

dupies = pd.DataFrame(dupe_q, columns =['Value', 'Doubled'])
dupies[dupies['Doubled'] == False] 

## This is not a short list! We will need to examine each of these and discuss with scientists!

Let's look at an example, use the index number to know how to pull up examples i.e, pull 5,9,10,13...

In [ ]:
len(list_of_data_frames[9])

In [ ]:
 How can we find how many repeaters?

In [ ]:
len(list_of_data_frames[9]) - len(list_of_data_frames[9].studieID.unique())

Unfortunately simple set math willl not help us here because the set of pandas column collapses to the unique values
As demonstrated below:

In [ ]:
len(list_of_data_frames[9].studieID) - len(set(list_of_data_frames[9].studieID))

So we will use filtering to find the duplicatd if we want it...demonstrated below:

In [ ]:
list_of_data_frames[9].studieID[list_of_data_frames[9].studieID.duplicated()]

## OK, on to seeing how many dupes there were per frame

In [ ]:
len(list_of_data_frames[5]) - len(list_of_data_frames[5].studieID.unique())

In [ ]:
len(list_of_data_frames[13]) - len(list_of_data_frames[13].studieID.unique())

In [ ]:
len(list_of_data_frames[22]) - len(list_of_data_frames[22].studieID.unique())

In [ ]:
len(list_of_data_frames[23]) - len(list_of_data_frames[23].studieID.unique())

In [ ]:
len(list_of_data_frames[24]) - len(list_of_data_frames[24].studieID.unique())

In [ ]:
len(list_of_data_frames[25]) - len(list_of_data_frames[25].studieID.unique())

In [ ]:
len(list_of_data_frames[26]) - len(list_of_data_frames[26].studieID.unique())

In [ ]:
len(list_of_data_frames[27]) - len(list_of_data_frames[27].studieID.unique())

## Scientists can do this by hand because the list is short....

# I suggest data engineers take over multiple possble transformations of data

In [ ]:
STOP

In [ ]:
final_df2.columns

In [ ]:
from functools import reduce

In [ ]:
#final_df = reduce(lambda left,right: pd.merge(left,right, on=['studieID'], how='outer'), list_of_data_frames)

In [ ]:
final_df

In [ ]:
final_df[final_df['studieID'] =='BRICK_001']

In [ ]:
for f in final_df.columns:
    print(f)

In [ ]:
final_df.to_csv('long_lab_data.csv')

In [ ]:
final_df.shape